In [1]:
import pandas as pd
import numpy as np

from pandas import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

import theano
import tensorflow

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


### Loading Dataset

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
X = df.iloc[:, 3:13].values
y = df.iloc[:, 13].values

### Examining Data

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


### Creating dummy variables for countries and transforming to array

In [5]:
lbl_ncdr_x_1 = LabelEncoder()
X[:, 1] = lbl_ncdr_x_1.fit_transform(X[:, 1])
lbl_ncdr_x_2 = LabelEncoder()
X[:, 2] = lbl_ncdr_x_2.fit_transform(X[:, 2])
hot_ncdr = OneHotEncoder(categorical_features=[1])
X = hot_ncdr.fit_transform(X).toarray()
X = X[:, 1:]

/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:385: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [6]:
X.shape

(10000, 11)

In [7]:
y.shape

(10000,)

### Creating train and test data

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Standardizing data

In [9]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

# Creating Classifier from Keras library

## Step 1: Adding layers

### Layer 1:

- using __add__ to add first hidden layer
- Adding 6 inputs, since our data has 11 columns
    - Using: ceil(# of columns/2) to get 6 layers
- Using rectifier function for hidden layers
- Input Dim is 11, since we have 11 variables and this is the first layer

### Layer 2:

- No need to add input dim, since it is not the first layer

### Layer 3:

- Using Sigmoid function for the output layer
- Since __y__ is binary, we only need one unit

## Step 2: Compiling ANN

- Using 'adam' optimizer
- Using 'binary_crossentropy' __(log loss)__ to calculate loss, since it is a logistic regression
- Using 'accuracy' as metrics for accuracy

In [10]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units=6, kernel_initializer='uniform',
                         activation='relu', input_dim=11))
    classifier.add(
        Dense(units=6, kernel_initializer='uniform', activation='relu'))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    classifier.compile(
        optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

### Running first model

In [24]:
# classifier.fit(X_train, y_train, batch_size=10, epochs=100)

### Creating k-fold  cross validation model

In [11]:
classifier = KerasClassifier(
    build_fn=build_classifier, batch_size=10, epochs=100)

### Running K-fold cross validation model

In [14]:
accs = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10)

Epoch 1/100
7200/7200 [==============================] - 2s 278us/step - loss: 0.4940 - acc: 0.7943
Epoch 2/100
7200/7200 [==============================] - 2s 217us/step - loss: 0.4208 - acc: 0.7946
Epoch 3/100
7200/7200 [==============================] - 2s 221us/step - loss: 0.4104 - acc: 0.8019
Epoch 4/100
7200/7200 [==============================] - 2s 226us/step - loss: 0.4011 - acc: 0.8268
Epoch 5/100
7200/7200 [==============================] - 2s 230us/step - loss: 0.3918 - acc: 0.8317
Epoch 6/100
7200/7200 [==============================] - 2s 223us/step - loss: 0.3847 - acc: 0.8337
Epoch 7/100
7200/7200 [==============================] - 2s 226us/step - loss: 0.3785 - acc: 0.8447
Epoch 8/100
7200/7200 [==============================] - 2s 223us/step - loss: 0.3739 - acc: 0.8487
Epoch 9/100
7200/7200 [==============================] - 2s 224us/step - loss: 0.3698 - acc: 0.8526
Epoch 10/100
7200/7200 [==============================] - 2s 225us/step - loss: 0.3673 - acc: 0.8503

7200/7200 [==============================] - 2s 216us/step - loss: 0.3476 - acc: 0.8574
Epoch 83/100
7200/7200 [==============================] - 2s 222us/step - loss: 0.3457 - acc: 0.8594
Epoch 84/100
7200/7200 [==============================] - 2s 238us/step - loss: 0.3457 - acc: 0.8594
Epoch 85/100
7200/7200 [==============================] - 2s 223us/step - loss: 0.3470 - acc: 0.8579
Epoch 86/100
7200/7200 [==============================] - 2s 227us/step - loss: 0.3467 - acc: 0.8599
Epoch 87/100
7200/7200 [==============================] - 2s 222us/step - loss: 0.3460 - acc: 0.8583
Epoch 88/100
7200/7200 [==============================] - 2s 233us/step - loss: 0.3466 - acc: 0.8585
Epoch 89/100
7200/7200 [==============================] - 2s 226us/step - loss: 0.3465 - acc: 0.8611
Epoch 90/100
7200/7200 [==============================] - 2s 238us/step - loss: 0.3465 - acc: 0.8593
Epoch 91/100
7200/7200 [==============================] - 2s 229us/step - loss: 0.3459 - acc: 0.8597
Epo

7200/7200 [==============================] - 2s 222us/step - loss: 0.3953 - acc: 0.8415
Epoch 63/100
7200/7200 [==============================] - 2s 225us/step - loss: 0.3956 - acc: 0.8410
Epoch 64/100
7200/7200 [==============================] - 2s 256us/step - loss: 0.3954 - acc: 0.8387
Epoch 65/100
7200/7200 [==============================] - 2s 320us/step - loss: 0.3953 - acc: 0.8390
Epoch 66/100
7200/7200 [==============================] - 3s 358us/step - loss: 0.3953 - acc: 0.8393
Epoch 67/100
7200/7200 [==============================] - 2s 230us/step - loss: 0.3952 - acc: 0.8382
Epoch 68/100
7200/7200 [==============================] - 2s 299us/step - loss: 0.3950 - acc: 0.8392
Epoch 69/100
7200/7200 [==============================] - 2s 276us/step - loss: 0.3951 - acc: 0.8403
Epoch 70/100
7200/7200 [==============================] - 2s 230us/step - loss: 0.3949 - acc: 0.8404
Epoch 71/100
7200/7200 [==============================] - 2s 226us/step - loss: 0.3947 - acc: 0.8386
Epo

7200/7200 [==============================] - 2s 235us/step - loss: 0.3933 - acc: 0.8418
Epoch 43/100
7200/7200 [==============================] - 2s 222us/step - loss: 0.3934 - acc: 0.8374
Epoch 44/100
7200/7200 [==============================] - 2s 224us/step - loss: 0.3930 - acc: 0.8382
Epoch 45/100
7200/7200 [==============================] - 2s 233us/step - loss: 0.3932 - acc: 0.8383
Epoch 46/100
7200/7200 [==============================] - 2s 231us/step - loss: 0.3930 - acc: 0.8400
Epoch 47/100
7200/7200 [==============================] - 2s 232us/step - loss: 0.3932 - acc: 0.8394
Epoch 48/100
7200/7200 [==============================] - 2s 233us/step - loss: 0.3929 - acc: 0.8371
Epoch 49/100
7200/7200 [==============================] - 2s 234us/step - loss: 0.3925 - acc: 0.8400
Epoch 50/100
7200/7200 [==============================] - 2s 238us/step - loss: 0.3929 - acc: 0.8382
Epoch 51/100
7200/7200 [==============================] - 2s 228us/step - loss: 0.3930 - acc: 0.8392
Epo

7200/7200 [==============================] - 2s 308us/step - loss: 0.3455 - acc: 0.8567
Epoch 23/100
7200/7200 [==============================] - 2s 240us/step - loss: 0.3444 - acc: 0.8568
Epoch 24/100
7200/7200 [==============================] - 2s 240us/step - loss: 0.3437 - acc: 0.8597
Epoch 25/100
7200/7200 [==============================] - 2s 246us/step - loss: 0.3437 - acc: 0.8585
Epoch 26/100
7200/7200 [==============================] - 3s 411us/step - loss: 0.3435 - acc: 0.8583
Epoch 27/100
7200/7200 [==============================] - 2s 304us/step - loss: 0.3432 - acc: 0.8599
Epoch 28/100
7200/7200 [==============================] - 2s 257us/step - loss: 0.3413 - acc: 0.8578
Epoch 29/100
7200/7200 [==============================] - 2s 266us/step - loss: 0.3436 - acc: 0.8601
Epoch 30/100
7200/7200 [==============================] - 2s 234us/step - loss: 0.3422 - acc: 0.8585
Epoch 31/100
7200/7200 [==============================] - 2s 243us/step - loss: 0.3419 - acc: 0.8583
Epo

...

7200/7200 [==============================] - 2s 267us/step - loss: 0.3471 - acc: 0.8582
Epoch 44/100
7200/7200 [==============================] - 2s 237us/step - loss: 0.3463 - acc: 0.8586
Epoch 45/100
7200/7200 [==============================] - 2s 266us/step - loss: 0.3465 - acc: 0.8585
Epoch 46/100
7200/7200 [==============================] - 2s 293us/step - loss: 0.3456 - acc: 0.8589
Epoch 47/100
7200/7200 [==============================] - 2s 271us/step - loss: 0.3451 - acc: 0.8576
Epoch 48/100
7200/7200 [==============================] - 2s 268us/step - loss: 0.3452 - acc: 0.8587
Epoch 49/100
7200/7200 [==============================] - 2s 297us/step - loss: 0.3442 - acc: 0.8601
Epoch 50/100
7200/7200 [==============================] - 2s 245us/step - loss: 0.3431 - acc: 0.8587
Epoch 51/100
7200/7200 [==============================] - 2s 251us/step - loss: 0.3429 - acc: 0.8599
Epoch 52/100
7200/7200 [==============================] - 2s 262us/step - loss: 0.3450 - acc: 0.8568
Epo

7200/7200 [==============================] - 2s 246us/step - loss: 0.4036 - acc: 0.8346
Epoch 24/100
7200/7200 [==============================] - 2s 241us/step - loss: 0.4034 - acc: 0.8351
Epoch 25/100
7200/7200 [==============================] - 3s 380us/step - loss: 0.4035 - acc: 0.8353
Epoch 26/100
7200/7200 [==============================] - 3s 373us/step - loss: 0.4030 - acc: 0.8354
Epoch 27/100
7200/7200 [==============================] - 2s 240us/step - loss: 0.4028 - acc: 0.8358
Epoch 28/100
7200/7200 [==============================] - 2s 281us/step - loss: 0.4025 - acc: 0.8342
Epoch 29/100
7200/7200 [==============================] - 3s 415us/step - loss: 0.4021 - acc: 0.8358
Epoch 30/100
7200/7200 [==============================] - 3s 381us/step - loss: 0.4020 - acc: 0.8356
Epoch 31/100
7200/7200 [==============================] - 2s 248us/step - loss: 0.4014 - acc: 0.8360
Epoch 32/100
7200/7200 [==============================] - 2s 338us/step - loss: 0.4016 - acc: 0.8350
Epo

7200/7200 [==============================] - 2s 288us/step - loss: 0.4212 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 2s 251us/step - loss: 0.4169 - acc: 0.8103
Epoch 5/100
7200/7200 [==============================] - 2s 250us/step - loss: 0.4130 - acc: 0.8279
Epoch 6/100
7200/7200 [==============================] - 2s 251us/step - loss: 0.4111 - acc: 0.8294
Epoch 7/100
7200/7200 [==============================] - 2s 258us/step - loss: 0.4093 - acc: 0.8329
Epoch 8/100
7200/7200 [==============================] - 2s 256us/step - loss: 0.4075 - acc: 0.8344
Epoch 9/100
7200/7200 [==============================] - 2s 259us/step - loss: 0.4064 - acc: 0.8360
Epoch 10/100
7200/7200 [==============================] - 2s 259us/step - loss: 0.4053 - acc: 0.8353
Epoch 11/100
7200/7200 [==============================] - 2s 261us/step - loss: 0.4041 - acc: 0.8358
Epoch 12/100
7200/7200 [==============================] - 2s 260us/step - loss: 0.4029 - acc: 0.8362
Epoch 13/

7200/7200 [==============================] - 2s 275us/step - loss: 0.3927 - acc: 0.8412
Epoch 85/100
7200/7200 [==============================] - 2s 249us/step - loss: 0.3931 - acc: 0.8403
Epoch 86/100
7200/7200 [==============================] - 3s 385us/step - loss: 0.3928 - acc: 0.8400
Epoch 87/100
7200/7200 [==============================] - 2s 270us/step - loss: 0.3932 - acc: 0.8400
Epoch 88/100
7200/7200 [==============================] - 2s 321us/step - loss: 0.3928 - acc: 0.8403
Epoch 89/100
7200/7200 [==============================] - 2s 291us/step - loss: 0.3933 - acc: 0.8394
Epoch 90/100
7200/7200 [==============================] - 2s 295us/step - loss: 0.3922 - acc: 0.8418
Epoch 91/100
7200/7200 [==============================] - 2s 268us/step - loss: 0.3926 - acc: 0.8403
Epoch 92/100
7200/7200 [==============================] - 2s 248us/step - loss: 0.3926 - acc: 0.8399
Epoch 93/100
7200/7200 [==============================] - 2s 272us/step - loss: 0.3927 - acc: 0.8412
Epo

7200/7200 [==============================] - 2s 260us/step - loss: 0.3977 - acc: 0.8392
Epoch 65/100
7200/7200 [==============================] - 2s 262us/step - loss: 0.3973 - acc: 0.8389
Epoch 66/100
7200/7200 [==============================] - 2s 248us/step - loss: 0.3974 - acc: 0.8367
Epoch 67/100
7200/7200 [==============================] - 2s 270us/step - loss: 0.3977 - acc: 0.8375
Epoch 68/100
7200/7200 [==============================] - 2s 258us/step - loss: 0.3974 - acc: 0.8374
Epoch 69/100
7200/7200 [==============================] - 2s 272us/step - loss: 0.3977 - acc: 0.8392
Epoch 70/100
7200/7200 [==============================] - 2s 267us/step - loss: 0.3969 - acc: 0.8383
Epoch 71/100
7200/7200 [==============================] - 2s 264us/step - loss: 0.3978 - acc: 0.8375
Epoch 72/100
7200/7200 [==============================] - 2s 263us/step - loss: 0.3975 - acc: 0.8385
Epoch 73/100
7200/7200 [==============================] - 2s 260us/step - loss: 0.3973 - acc: 0.8381
Epo

### Accuracies from k-fold cross validation

In [15]:
accs

array([0.86624999, 0.82625   , 0.86124999, 0.86624999, 0.8275    ,
       0.8075    , 0.85499999, 0.85499999, 0.82125   , 0.84125   ])

### Taking the mean of all k-fold cross validation models to get an average from all tests

In [18]:
accu_mean = accs.mean()

In [21]:
accu_mean

0.8427499949559569

### Checking variance of all k-fold cross validation models

In [22]:
accu_var = accs.std()

In [23]:
accu_var

0.01989660622203791

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
y_pred

In [ ]:
cm

In [ ]:
print(f'Accuracy = {((1526+193)/20)}%')